In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-midterm-p5/train.csv
/kaggle/input/2022-ai-midterm-p5/test.csv
/kaggle/input/2022-ai-midterm-p5/submit_sample.csv


In [2]:
#랜덤시드 고정
import random
import torch

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022-ai-midterm-p5/submit_sample.csv')
train = pd.read_csv('/kaggle/input/2022-ai-midterm-p5/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-midterm-p5/test.csv')

In [4]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
#데이터 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   15326 non-null  int64  
 1   enrollee_id             15326 non-null  int64  
 2   city                    15326 non-null  object 
 3   city_development_index  15326 non-null  float64
 4   gender                  11752 non-null  object 
 5   relevent_experience     15326 non-null  object 
 6   enrolled_university     15009 non-null  object 
 7   education_level         14964 non-null  object 
 8   major_discipline        13068 non-null  object 
 9   experience              15276 non-null  object 
 10  company_size            10612 non-null  object 
 11  company_type            10445 non-null  object 
 12  last_new_job            14983 non-null  object 
 13  training_hours          15326 non-null  int64  
 14  target                  15326 non-null

In [6]:
train

,index,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
3,4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
4,5,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15321,19152,29754,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,Humanities,7,10/49,Funded Startup,1,25,0.0
15322,19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
15323,19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
15324,19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [7]:
#데이터 전처리
xtrain = train.drop(['index','enrollee_id','target'],axis=1)
ytrain = train['target']
xtest = test.drop(['index','enrollee_id'],axis=1)

In [8]:
#데이터의 column들을 모두 읽어오기
xtrain.columns

Index(['city', 'city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job',
       'training_hours'],
      dtype='object')

In [9]:
#결측값 확인
label = ['city', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job']
for i in label:
    print(xtrain[i].unique())

['city_103' 'city_40' 'city_115' 'city_162' 'city_176' 'city_160'
 'city_46' 'city_61' 'city_21' 'city_13' 'city_159' 'city_102' 'city_67'
 'city_100' 'city_16' 'city_114' 'city_71' 'city_104' 'city_64' 'city_101'
 'city_83' 'city_105' 'city_73' 'city_75' 'city_41' 'city_11' 'city_93'
 'city_90' 'city_36' 'city_57' 'city_19' 'city_65' 'city_74' 'city_97'
 'city_50' 'city_138' 'city_82' 'city_157' 'city_89' 'city_70' 'city_173'
 'city_98' 'city_94' 'city_136' 'city_59' 'city_165' 'city_145' 'city_28'
 'city_142' 'city_26' 'city_12' 'city_37' 'city_43' 'city_116' 'city_23'
 'city_99' 'city_149' 'city_152' 'city_150' 'city_10' 'city_45' 'city_80'
 'city_158' 'city_123' 'city_72' 'city_20' 'city_143' 'city_78' 'city_109'
 'city_24' 'city_134' 'city_48' 'city_144' 'city_91' 'city_146' 'city_126'
 'city_128' 'city_175' 'city_118' 'city_9' 'city_167' 'city_84' 'city_54'
 'city_39' 'city_76' 'city_77' 'city_81' 'city_106' 'city_44' 'city_117'
 'city_155' 'city_33' 'city_141' 'city_27' 'city_12

In [10]:
#object형의 데이터들을 int형으로 전처리
from sklearn import preprocessing
for i in label:
    box = np.concatenate([xtrain[i].unique(),xtest[i].unique()])
    le = preprocessing.LabelEncoder()
    le.fit(box)
    xtrain[i] = le.transform(xtrain[i])
    xtest[i] = le.transform(xtest[i])

In [11]:
#데이터 전처리
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

xtrain_std = sc.fit_transform(xtrain)
xtest_std = sc.transform(xtest)

In [12]:
# X_train = torch.FloatTensor(np.array(xtrain)).to(device)
# y_train = torch.FloatTensor(np.array(ytrain)).to(device)
# X_test = torch.FloatTensor(np.array(xtest)).to(device)
X_train = torch.FloatTensor(np.array(xtrain_std)).to(device)
y_train = torch.FloatTensor(np.array(ytrain)).to(device)
X_test = torch.FloatTensor(np.array(xtest_std)).to(device)

In [13]:
#모델 빌드를 위한 설정값 확인
print(X_train.shape)
print(y_train.shape)

torch.Size([15326, 12])
torch.Size([15326])


In [14]:
#모델 빌드
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(12, 512, bias=True)
        self.linear2 = nn.Linear(512, 128, bias=True)
        self.linear3 = nn.Linear(128, 1, bias=True)
        
        self.sigmoid = nn.Sigmoid()
        self.active = nn.ReLU()
        
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.active(x)
        x = self.linear2(x)
        x = self.active(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        
        return x

In [15]:
#모델 정의
model = DNN().to(device)

In [16]:
import torch.optim as optim

#Optimizer 정의
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#Loss 정의 (이진분류이므로 BCELoss)
loss = nn.BCELoss()

In [17]:
#모델 학습
epochs = 150
for epoch in range(epochs + 1):
    
    optimizer.zero_grad()
    
    hypothesis = model(X_train)
    
    cost = loss(hypothesis, y_train.unsqueeze(1))
    
    cost.backward()
    
    optimizer.step()
    
    if epoch % 10 == 0:
        print(epoch, cost.item())

0 0.7412141561508179
10 0.5018192529678345
20 0.47696420550346375
30 0.467632919549942
40 0.46196508407592773
50 0.4567777216434479
60 0.4526005983352661
70 0.4491020441055298
80 0.44586655497550964
90 0.442792147397995
100 0.4398626685142517
110 0.43701037764549255
120 0.4341631531715393
130 0.4312676191329956
140 0.42832663655281067
150 0.42527565360069275


In [18]:
#학습한 모델을 이용해서 Test 데이터 예측
with torch.no_grad():
    model.eval()
    hypothesis = model(X_test)
    predict = (hypothesis >= torch.FloatTensor([0.5]).to(device)).to(torch.float64)

In [19]:
#예측한 값을 csv로 변환
sample['target'] = predict.cpu().detach().numpy()
sample.to_csv('submit.csv',index=False)